In [12]:
# from langchain_
import os
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
# os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")
# os.environ["ALPHA_VANTAGE_API_KEY"] = os.getenv("ALPHA_VANTAGE_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2", "true")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")



In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain.agents import initialize_agent, AgentType,load_tools
from langchain_ollama import OllamaEmbeddings

embeddings =OllamaEmbeddings(model="mistral")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [15]:
# res=embeddings("Hello world")
# res

In [16]:
from langchain_groq import ChatGroq

llm_groq = ChatGroq(model="Gemma2-9b-It", groq_api_key=os.getenv("GROQ_API_KEY"))

In [17]:
class Chatbot():
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash", 
            google_api_key=os.getenv("GOOGLE_API_KEY")
        )
        result = llm.invoke(self.messages)
        return result.content


    # def get_llm(self):
    #     return self.llm

    # def get_llm_groq(self):
    #     return self.llm_groq

    # def get_embeddings(self):
    #     return self.embeddings

    # def set_llm(self, llm):
    #     self.llm = llm

    # def set_llm_groq(self, llm_groq):
    #     self.llm_groq = llm_groq

    # def set_embeddings(self, embeddings):
    #     self.embeddings = embeddings

In [18]:
bot=Chatbot(system="You are a helpful assistant.")

In [19]:
bot("Hello, how are you?")

'I am doing well, thank you for asking! How can I help you today?'

In [20]:
# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop your output an Answer.
# Use Thought to describe your thoughts about the question you have been asked.
# Use Action to run one of the actions available to you - then return PAUSE.
# Observation will be the result of running those actions.


# Your available actions are:
# calculate:
# e.g. calculate: 4 * 7 / 3
# Runs a calculation and returns the number - uses Python so be sure to use floating point
# syntax if necessary

# wikipedia:
# e.g. wikipedia: Django
# Returns a summary from searching Wikipedia

# simon_blog_search:
# e.g. simon_blog_search: Python
# Search Simon's blog for that term

# Example session:
# Question: What is the capital of France?
# Thought: I should look up France on Wikipedia
# Action: wikipedia: France
# PAUSE

# You will be called again with this:
# Observation: France is a country. The capital is Paris.

# You then output:
# Answer: The capital of France is Paris

# Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
# you have to answer "hi","hello","i am good".
# """.strip()


prompt = """
You operate in a cycle of Thought, Action, PAUSE, and Observation.
After Observation, you output an Answer and stop.

• Thought: Describe your reasoning about the user’s question.
• Action: Call one of these functions, then immediately return PAUSE.
• Observation: The result of that function call.

Available functions:
- calculate:
    e.g. calculate: 4 * 7 / 3
    Runs a Python calculation and returns the numeric result.

- wikipedia:
    e.g. wikipedia: Django
    Returns a two‑sentence summary from Wikipedia.

- search_tavily:
    e.g. search_tavily: Python
    Searches web using the tavily and returns the top result.

- get_stock_data:
    e.g. get_stock_data: RELIANCE.NS
    Fetches the latest closing price for an Indian stock ticker on Yahoo Finance.
    • Use “.NS” for National Stock Exchange (NSE) symbols.
    • Use “.BO” for Bombay Stock Exchange (BSE) symbols. :contentReference[oaicite:0]{index=0}

Example session:

Question: What is the current price of Asian Paints?  
Thought: I need to find the latest closing price of Asian Paints on NSE.  
Action: get_stock_data: ASIANPAINT.NS  pass the ticker symbol to the tool
PAUSE : check the output of the tool


When called again with the tool output:

Observation: ASIANPAINT.NS closed at ₹3,115.20

You then output:

Answer: The current closing price of Asian Paints on NSE is ₹3,115.20.

Note:  
For simple greetings like “hi”, “hello”, or “how are you?”, respond directly with “Hi”, “Hello”, or “I am good” (no tools needed).
""".strip()



In [21]:
import re
# action_re = re.compile('^Action: (\w+): (.*)')
action_re = re.compile(r"Action:\s*(\w+)\s*:\s*(.+)")


In [22]:
import httpx

def wikipedia(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "format": "json",
        "list": "search",
        "srsearch": query,
        "utf8": 1,
        "srlimit": 1,
    }
    response = httpx.get(url, params=params)
    # data = response.json()
    # if 'query' in data and 'search' in data['query']:
    return response.json()['query']['search'][0]['snippet']
    # else:
        # return None

# wikipedia("Python")

In [23]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [24]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [25]:
import yfinance as yf

def get_indian_stock_data(
    ticker_symbol: str,
    period: str = "5d",
    interval: str = "1d",
    info_fields: list[str] | None = None,
    news_count: int = 5
) -> dict:
    """
    Fetch Indian stock data and metadata using yfinance.

    Args:
        ticker_symbol (str): Ticker with exchange suffix (e.g., 'RELIANCE.NS').
        period (str): Historical data period (e.g., '1d','5d','1mo','1y').
        interval (str): Historical data interval (e.g., '1d','1h','5m').
        info_fields (list[str] | None): Which info keys to include. Defaults to key metrics.
        news_count (int): Number of top news items to return.

    Returns:
        dict: {
            'historical': DataFrame,
            'info': { ... },
            'news': [ ... ]
        }
    """
    ticker = yf.Ticker(ticker_symbol)
    # 1) Historical prices
    hist = ticker.history(period=period, interval=interval)

    # 2) Company info (select fields or default set)
    default_fields = [
        "shortName", "longName",
        "previousClose", "open", "dayHigh", "dayLow",
        "marketCap", "trailingPE", "forwardPE",
        "dividendYield",
        "recommendationKey", "targetMeanPrice",
        "targetHighPrice", "targetLowPrice"
    ]
    info = ticker.info or {}
    chosen = info_fields or default_fields
    info_data = {k: info.get(k) for k in chosen}

    # 3) Latest news
    raw_news = getattr(ticker, "news", []) or []
    news = raw_news[:news_count]

    return {
        "historical": hist,
        "info": info_data,
        "news": news,
    }




In [33]:
df = get_indian_stock_data("ADANIENT.NS", period="1mo", interval="1d")
print(df)


{'historical':                                   Open         High          Low        Close  \
Date                                                                            
2025-03-04 00:00:00+05:30  2085.100098  2154.000000  2080.000000  2144.800049   
2025-03-05 00:00:00+05:30  2153.949951  2258.399902  2148.050049  2245.850098   
2025-03-06 00:00:00+05:30  2260.000000  2268.899902  2226.149902  2252.850098   
2025-03-07 00:00:00+05:30  2249.949951  2267.649902  2227.000000  2247.500000   
2025-03-10 00:00:00+05:30  2245.000000  2288.000000  2220.000000  2228.300049   
2025-03-11 00:00:00+05:30  2223.149902  2267.850098  2208.300049  2250.600098   
2025-03-12 00:00:00+05:30  2270.000000  2280.000000  2180.949951  2241.800049   
2025-03-13 00:00:00+05:30  2249.000000  2273.000000  2215.600098  2221.100098   
2025-03-17 00:00:00+05:30  2200.000000  2286.899902  2198.000000  2252.699951   
2025-03-18 00:00:00+05:30  2252.699951  2252.699951  2252.699951  2252.699951   
2025-03-19 00

In [27]:
def calculate(number):
    return eval(number)

In [28]:
# # action_pattern = re.compile(r"Action:\s*(\w+)\s*:\s*(.+)")
import httpx
import os
from dotenv import load_dotenv
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
def search_tavily(query):
    response = httpx.post('https://api.tavily.com/search',json={
         "query": query,
        "topic": "general",
        "search_depth": "basic",
        "chunks_per_source": 3,
        "max_results": 1,
    },headers={
        "Authorization": f"Bearer {TAVILY_API_KEY}",
        "Content-Type": "application/json"
    })
    return response.json()['results'][0]['content']
# search("reliance stock current price")

In [29]:
# result=search_tavily("Who saved virat kohli's carrer?")


In [30]:


# 1) Register all your tools here:
known_actions = {
    "calculate": calculate,
    "wikipedia": wikipedia,
    "tavily_search": search_tavily,
    # register your stock‐data function under the same name the agent uses:
    "get_stock_data": get_indian_stock_data,  
}


def query(question: str, max_turns: int = 10) -> str:
    bot = Chatbot(prompt)
    user_input = question

    for turn in range(1, max_turns + 1):
        response = bot(user_input)
        print(f"Turn {turn} response:\n{response}\n")

        # If the agent asked to pause for a tool…
        if "PAUSE" in response:
            m = action_re.search(response)
            if not m:
                raise ValueError("PAUSE detected but no Action found.")
            action_name, action_arg = m.groups()

            if action_name not in known_actions:
                raise ValueError(f"Unknown action: {action_name}")

            print(f"→ Running tool {action_name}('{action_arg}')")
            observation = known_actions[action_name](action_arg)
            print(f"Observation:\n{observation}\n")

            user_input = f"Observation: {observation}"
            continue

        # No PAUSE means final Answer
        return response

    raise RuntimeError("Max turns exceeded without final Answer.")


In [31]:




# def query(question,max_turns=5):
#     i=0
#     bot = Chatbot(prompt)
#     next_prompt=question
#     while i<max_turns:
#         i+=1
#         result = bot.execute(next_prompt)
#         print(f"Bot: {result}")
#         action = action_re.match(result)
#         if action:
#             action_name, action_input = action.groups()
#             if action_name in known_actions:
#                 print(f"Action: {action_name} with input: {action_input}")
#                 next_prompt = known_actions[action_name](action_input)
#             else:
#                 print(f"Unknown action: {action_name}")
#                 break
#         else:
#             print("No action found in the response.")
#             break
#     return result

# def query(question,max_turns=50):
#     i = 0
#     bot = Chatbot(prompt)
#     next_prompt = question
#     while i < max_turns:
#         i += 1
#         result = bot(next_prompt)
#         print("1",result)
#         actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
#         if actions:
#             action, action_input = actions[0].groups()
#             if action not in known_actions:
#                 raise Exception(f"Unknown action: {action}: {action_input}")
#             print(" -- running {} {}".format(action, action_input))
#             observation = known_actions[action](action_input)
#             print("2 Observation:", observation)
#             next_prompt = f"Observation: {observation}"
#         else:
#             return result

In [36]:
bot = Chatbot(prompt)
query("Generate code for Static html page of the content with hi harshit ")


Turn 1 response:
I am sorry, I cannot generate code directly. However, I can provide you with the HTML code for a static page containing the text "Hi Harshit".

```html
<!DOCTYPE html>
<html>
<head>
<title>Simple HTML Page</title>
</head>
<body>

<h1>Hi Harshit</h1>

</body>
</html>
```

You can copy and paste this code into a text file, save it with a `.html` extension (e.g., `index.html`), and then open it in a web browser.



'I am sorry, I cannot generate code directly. However, I can provide you with the HTML code for a static page containing the text "Hi Harshit".\n\n```html\n<!DOCTYPE html>\n<html>\n<head>\n<title>Simple HTML Page</title>\n</head>\n<body>\n\n<h1>Hi Harshit</h1>\n\n</body>\n</html>\n```\n\nYou can copy and paste this code into a text file, save it with a `.html` extension (e.g., `index.html`), and then open it in a web browser.'